# Funding Timeframes

How often to organizations apply for grants?

- this notebook takes a look at the DLA data application 

In [1]:
import pandas as pd
from siuba import *
import numpy as np

In [2]:
from IPython.display import Markdown

In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 1000)

In [4]:
from calitp.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/'

pd.set_option('display.max_columns', None)


/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
df = pd.read_parquet(f"{GCS_FILE_PATH}dla_df.parquet") 

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [6]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
17057,Obligated,CML,5165(084),Turlock,2020-06-08,2020-06-10,2020-08-02,2020-08-02,2020-08-05,219191.0,0.0,219191.0,Authorized,5165,10,E-76 approved on,5.0,53.0,3.0,STANCOG,Lander Avenue At F Street,"Installation Of Traffic Signal, Pedestrian Ramps, Striping, And Various Related",3,2020-06-05,2020-06-10,STANCOG,5165,2020.0,Turlock,229488.506918,229488.506918,0.0,1,0,0,1,0,0,0,2


In [7]:
t = df >> select(
    _.primary_agency_name,
    _.locode,
    _.project_no,
    _.prefix,
    _.prepared_date,
    _.prepared_y,
)

In [8]:
t["prepared_m"] = pd.DatetimeIndex(t["prepared_date"]).month

In [9]:
t = t>>filter(_.prepared_date.notnull())

In [10]:
t

,primary_agency_name,locode,project_no,prefix,prepared_date,prepared_y,prepared_m
0,Humboldt County,5904,5904(121),BPMP,2018-12-18,2018.0,12.0
1,Mendocino County,5910,32D0(008),ER,2018-12-17,2018.0,12.0
2,Humboldt County,5904,4820(004),ER,2018-12-07,2018.0,12.0
3,Sacramento County,5924,5924(244),CML,2018-12-11,2018.0,12.0
4,Sacramento County,5924,5924(214),CML,2018-12-05,2018.0,12.0
...,...,...,...,...,...,...,...
21643,Morongo Basin Transit Authority,6235,6235(021),FTACML,2022-02-18,2022.0,2.0
21647,Santa Clara Valley Transportation Authority,6264,6264(091),FTASTPL,2021-11-22,2021.0,11.0
21648,Alameda - Contra Costa Transit District,6002,6002(030),FTASTPL,2021-11-22,2021.0,11.0
21649,Mountain Area Regional Transit Authority,6292,6292(016),FTACML,2022-02-18,2022.0,2.0


In [11]:
(t>>group_by(_.prepared_y, _.primary_agency_name)>>count(_.primary_agency_name)>>arrange(-_.n)).head(10)

,prepared_y,primary_agency_name,n
2100,2018.0,Humboldt County,171
2537,2019.0,Humboldt County,147
2305,2018.0,Santa Cruz County,109
2730,2019.0,Santa Cruz County,105
330,2014.0,Los Angeles,86
2170,2018.0,Modesto,85
2947,2020.0,Humboldt County,84
726,2015.0,Fresno,83
2578,2019.0,Los Angeles County,74
1120,2016.0,Caltrans,73


In [12]:
t2 =(
    t
    >> group_by(_.primary_agency_name, _.prepared_y)
    >> count(_.prepared_m)
#    >> filter(_.prepared_y == 2018)
    >> arrange(_.primary_agency_name, _.prepared_y, _.prepared_m)
)

In [13]:
t2.head(20)

,primary_agency_name,prepared_y,prepared_m,n
0,Access Services,2014.0,2.0,1
1,Access Services,2015.0,3.0,2
2,Access Services,2016.0,3.0,7
3,Access Services,2017.0,3.0,1
4,Access Services,2018.0,3.0,1
5,Access Services,2019.0,4.0,2
6,Access Services,2019.0,12.0,1
7,Access Services,2020.0,2.0,1
8,Access Services,2021.0,2.0,1
9,Access Services,2022.0,3.0,1


In [14]:
(t2>>filter(_.prepared_y==2018)>>filter(_.primary_agency_name=='Alameda County'))

,primary_agency_name,prepared_y,prepared_m,n
65,Alameda County,2018.0,1.0,2
66,Alameda County,2018.0,2.0,3
67,Alameda County,2018.0,3.0,1
68,Alameda County,2018.0,4.0,4
69,Alameda County,2018.0,5.0,1
70,Alameda County,2018.0,6.0,1
71,Alameda County,2018.0,8.0,1
72,Alameda County,2018.0,10.0,2


In [15]:
((t2>>filter(_.prepared_y==2018)) >> group_by(_.primary_agency_name) >> summarize(n=_.prepared_m.nunique()) >> arrange(-_.n))

,primary_agency_name,n
185,Los Angeles,12
186,Los Angeles County,12
19,Bakersfield,11
99,El Dorado County,11
140,Humboldt County,11
...,...,...
418,Waterford,1
419,Watsonville,1
426,Willows,1
428,Winters,1


### Days Between Dates
from https://towardsdatascience.com/all-the-pandas-shift-you-should-know-for-data-analysis-791c1692b5e

In [16]:
t

,primary_agency_name,locode,project_no,prefix,prepared_date,prepared_y,prepared_m
0,Humboldt County,5904,5904(121),BPMP,2018-12-18,2018.0,12.0
1,Mendocino County,5910,32D0(008),ER,2018-12-17,2018.0,12.0
2,Humboldt County,5904,4820(004),ER,2018-12-07,2018.0,12.0
3,Sacramento County,5924,5924(244),CML,2018-12-11,2018.0,12.0
4,Sacramento County,5924,5924(214),CML,2018-12-05,2018.0,12.0
...,...,...,...,...,...,...,...
21643,Morongo Basin Transit Authority,6235,6235(021),FTACML,2022-02-18,2022.0,2.0
21647,Santa Clara Valley Transportation Authority,6264,6264(091),FTASTPL,2021-11-22,2021.0,11.0
21648,Alameda - Contra Costa Transit District,6002,6002(030),FTASTPL,2021-11-22,2021.0,11.0
21649,Mountain Area Regional Transit Authority,6292,6292(016),FTACML,2022-02-18,2022.0,2.0


In [17]:
t3 = (
    t
    >> filter(_.primary_agency_name == "Santa Cruz County")
    >> select(_.primary_agency_name, _.prepared_date, _.prepared_y, _.prepared_m)
    >> arrange(_.prepared_date)
)

In [18]:
t3

,primary_agency_name,prepared_date,prepared_y,prepared_m
12604,Santa Cruz County,2014-02-06,2014.0,2.0
12564,Santa Cruz County,2014-02-27,2014.0,2.0
12337,Santa Cruz County,2014-03-19,2014.0,3.0
12424,Santa Cruz County,2014-03-25,2014.0,3.0
12121,Santa Cruz County,2014-04-16,2014.0,4.0
...,...,...,...,...
20853,Santa Cruz County,2022-02-15,2022.0,2.0
21430,Santa Cruz County,2022-02-17,2022.0,2.0
21454,Santa Cruz County,2022-03-02,2022.0,3.0
21013,Santa Cruz County,2022-03-08,2022.0,3.0


In [19]:
t3

,primary_agency_name,prepared_date,prepared_y,prepared_m
12604,Santa Cruz County,2014-02-06,2014.0,2.0
12564,Santa Cruz County,2014-02-27,2014.0,2.0
12337,Santa Cruz County,2014-03-19,2014.0,3.0
12424,Santa Cruz County,2014-03-25,2014.0,3.0
12121,Santa Cruz County,2014-04-16,2014.0,4.0
...,...,...,...,...
20853,Santa Cruz County,2022-02-15,2022.0,2.0
21430,Santa Cruz County,2022-02-17,2022.0,2.0
21454,Santa Cruz County,2022-03-02,2022.0,3.0
21013,Santa Cruz County,2022-03-08,2022.0,3.0


In [20]:
t3['days_between'] = (t3['prepared_date'] - t3.shift(1)['prepared_date']).dt.days

In [21]:
t3

,primary_agency_name,prepared_date,prepared_y,prepared_m,days_between
12604,Santa Cruz County,2014-02-06,2014.0,2.0,NaN
12564,Santa Cruz County,2014-02-27,2014.0,2.0,21.0
12337,Santa Cruz County,2014-03-19,2014.0,3.0,20.0
12424,Santa Cruz County,2014-03-25,2014.0,3.0,6.0
12121,Santa Cruz County,2014-04-16,2014.0,4.0,22.0
...,...,...,...,...,...
20853,Santa Cruz County,2022-02-15,2022.0,2.0,6.0
21430,Santa Cruz County,2022-02-17,2022.0,2.0,2.0
21454,Santa Cruz County,2022-03-02,2022.0,3.0,13.0
21013,Santa Cruz County,2022-03-08,2022.0,3.0,6.0


In [22]:
t6 =(
    t3
    >> group_by(_.primary_agency_name, _.prepared_y)
    >> summarize(avg_days=_.days_between.mean(),
                obs = _.prepared_date.count())
)

In [23]:
t6

,primary_agency_name,prepared_y,avg_days,obs
0,Santa Cruz County,2014.0,12.619048,22
1,Santa Cruz County,2015.0,14.482759,29
2,Santa Cruz County,2016.0,17.900000,20
3,Santa Cruz County,2017.0,4.916667,72
4,Santa Cruz County,2018.0,3.165138,109
5,Santa Cruz County,2019.0,3.819048,105
6,Santa Cruz County,2020.0,6.048387,62
7,Santa Cruz County,2021.0,8.325581,43
8,Santa Cruz County,2022.0,4.722222,18


## Creating Loop for Some Agencies

In [29]:
agencies = [
    "Santa Cruz County",
    "Humboldt County",
    "Los Angeles",
    "Modesto",
    "Fresno",
    "Los Angeles County",
    "Caltrans"
]

for i in agencies:
    t4 = (
        t
        >> filter(_.primary_agency_name == i)
        >> select(_.primary_agency_name, _.prepared_date, _.prepared_y, _.prepared_m)
        >> arrange(_.prepared_date)
    )
    t4["days_between"] = (t4["prepared_date"] - t4.shift(1)["prepared_date"]).dt.days
    t4 = (
        t4
        >> group_by(_.primary_agency_name, _.prepared_y)
        >> summarize(avg_days=_.days_between.mean(),
                     number = _.prepared_date.count())
    )
    
    display(Markdown(f'**Average Days Between Projects in {i}**'))
    display(t4)

**Average Days Between Projects in Santa Cruz County**

,primary_agency_name,prepared_y,avg_days,number
0,Santa Cruz County,2014.0,12.619048,22
1,Santa Cruz County,2015.0,14.482759,29
2,Santa Cruz County,2016.0,17.900000,20
3,Santa Cruz County,2017.0,4.916667,72
4,Santa Cruz County,2018.0,3.165138,109
5,Santa Cruz County,2019.0,3.819048,105
6,Santa Cruz County,2020.0,6.048387,62
7,Santa Cruz County,2021.0,8.325581,43
8,Santa Cruz County,2022.0,4.722222,18


**Average Days Between Projects in Humboldt County**

,primary_agency_name,prepared_y,avg_days,number
0,Humboldt County,2013.0,NaN,1
1,Humboldt County,2014.0,7.176471,51
2,Humboldt County,2015.0,6.865385,52
3,Humboldt County,2016.0,11.030303,33
4,Humboldt County,2017.0,12.214286,28
5,Humboldt County,2018.0,2.269006,171
6,Humboldt County,2019.0,2.476190,147
7,Humboldt County,2020.0,4.476190,84
8,Humboldt County,2021.0,4.816901,71
9,Humboldt County,2022.0,1.277778,72


**Average Days Between Projects in Los Angeles**

...

,primary_agency_name,prepared_y,avg_days,number
0,Fresno,2013.0,18.333333,4
1,Fresno,2014.0,6.500000,58
2,Fresno,2015.0,4.301205,83
3,Fresno,2016.0,6.200000,60
4,Fresno,2017.0,5.625000,56
5,Fresno,2018.0,6.960784,51
6,Fresno,2019.0,6.655738,61
7,Fresno,2020.0,7.720000,50
8,Fresno,2021.0,9.324324,37
9,Fresno,2022.0,11.250000,8


**Average Days Between Projects in Los Angeles County**

,primary_agency_name,prepared_y,avg_days,number
0,Los Angeles County,2013.0,56.600000,6
1,Los Angeles County,2014.0,6.313433,67
2,Los Angeles County,2015.0,9.097561,41
3,Los Angeles County,2016.0,9.421053,38
4,Los Angeles County,2017.0,6.228070,57
5,Los Angeles County,2018.0,5.625000,64
6,Los Angeles County,2019.0,5.094595,74
7,Los Angeles County,2020.0,5.267606,71
8,Los Angeles County,2021.0,7.458333,48
9,Los Angeles County,2022.0,5.500000,16


**Average Days Between Projects in Caltrans**

,primary_agency_name,prepared_y,avg_days,number
0,Caltrans,2015.0,6.111111,10
1,Caltrans,2016.0,5.013699,73
2,Caltrans,2017.0,9.571429,14
3,Caltrans,2021.0,536.666667,3
4,Caltrans,2022.0,56.000000,3


## Shift For All Agencies 

In [29]:
t

,primary_agency_name,locode,project_no,prefix,prepared_date,prepared_y,prepared_m
0,Humboldt County,5904,5904(121),BPMP,2018-12-18,2018.0,12.0
1,Mendocino County,5910,32D0(008),ER,2018-12-17,2018.0,12.0
2,Humboldt County,5904,4820(004),ER,2018-12-07,2018.0,12.0
3,Sacramento County,5924,5924(244),CML,2018-12-11,2018.0,12.0
4,Sacramento County,5924,5924(214),CML,2018-12-05,2018.0,12.0
...,...,...,...,...,...,...,...
21643,Morongo Basin Transit Authority,6235,6235(021),FTACML,2022-02-18,2022.0,2.0
21647,Santa Clara Valley Transportation Authority,6264,6264(091),FTASTPL,2021-11-22,2021.0,11.0
21648,Alameda - Contra Costa Transit District,6002,6002(030),FTASTPL,2021-11-22,2021.0,11.0
21649,Mountain Area Regional Transit Authority,6292,6292(016),FTACML,2022-02-18,2022.0,2.0


In [42]:
sort_cols = ["primary_agency_name", "prepared_date"]
group_cols = ["primary_agency_name"]


In [50]:
shift = t.assign(
    previous_date=(
        t.sort_values(sort_cols)
        .groupby(group_cols)['prepared_date']
        .apply(lambda x: x.shift(1))
    )
)

In [51]:
#checking to see if it goes over to the next agency
(shift>>arrange(_.primary_agency_name, _.prepared_date)).head(20)

,primary_agency_name,locode,project_no,prefix,prepared_date,prepared_y,prepared_m,previous_date
12613,Access Services,6312,6312(019),FTASTPL,2014-02-24,2014.0,2.0,NaT
9663,Access Services,6312,6312(020),FTASTPL,2015-03-16,2015.0,3.0,2014-02-24
9664,Access Services,6312,6312(020),FTASTPL,2015-03-16,2015.0,3.0,2015-03-16
7096,Access Services,6312,6312(022),FTASTPL,2016-03-02,2016.0,3.0,2015-03-16
7097,Access Services,6312,6312(022),FTASTPL,2016-03-02,2016.0,3.0,2016-03-02
7098,Access Services,6312,6312(022),FTASTPL,2016-03-02,2016.0,3.0,2016-03-02
7099,Access Services,6312,6312(022),FTASTPL,2016-03-02,2016.0,3.0,2016-03-02
7192,Access Services,6312,6312(022),FTASTPL,2016-03-02,2016.0,3.0,2016-03-02
7193,Access Services,6312,6312(022),FTASTPL,2016-03-02,2016.0,3.0,2016-03-02
7194,Access Services,6312,6312(022),FTASTPL,2016-03-02,2016.0,3.0,2016-03-02


In [54]:
shift["days_between"] = (shift["prepared_date"] - shift["previous_date"]).dt.days

In [57]:
shift.days_between.describe()

count    20407.000000
mean        68.143088
std        147.252365
min          0.000000
25%          3.000000
50%         17.000000
75%         66.000000
max       2366.000000
Name: days_between, dtype: float64

In [63]:
(
    shift
    >> group_by(_.primary_agency_name)
    >> summarize(avg_app=_.days_between.mean())
    >> filter(_.avg_app.notnull())
    >> arrange(_.avg_app)
).head(20)

,primary_agency_name,avg_app
16,Amador Rapid Transit System,0.000000
564,Tuolumne County Transportation Council,0.000000
210,Humboldt County,4.218618
276,Los Angeles,6.081349
486,Santa Cruz County,6.181628
175,Fresno,6.548180
277,Los Angeles County,6.962578
149,El Dorado County,10.067742
228,Kern County,10.084746
458,San Joaquin County,10.918819
